# **Procesamiento de PDFs BOE y creación de base de datos vectorial FAISS**

### Estructura del RAG

```
Pregunta del usuario
         │
         ▼
  ┌───────────────┐
  │ Retriever     │  <- FAISS + SentenceTransformer
  │ (embeddings)  │
  └───────────────┘
         │  (chunks más relevantes)
         ▼
  ┌─────────────────────────┐
  │ Traducción al inglés    │  <- MarianMT
  └─────────────────────────┘
         │
         ▼
  ┌───────────────┐
  │ Generator     │  <- BART (resumen en inglés)
  │ (LLM)         │
  └───────────────┘
         │
         ▼
  ┌─────────────────────────┐
  │ Traducción al español   │  <- MarianMT
  └─────────────────────────┘
         │
         ▼
Respuesta RAG + citas de los documentos

```


**A) Retriever**

- Modelo: `all-MiniLM-L6-v2`
  
- Función: `retrieve(query, top_k=N)`

- Base de datos: FAISS con embeddings de los chunks de los PDFs

- Salida: lista de chunks relevantes con `doc_id`, `chunk_id` y `text`

**B) Chunking**

- Cada documento se divide en fragmentos de ~500 caracteres (`chunk_text`)

- Esto ayuda a que los embeddings sean más precisos y a no superar los límites de contexto de los modelos de generación

- También permite traducir fragmentos sin exceder los límites de los modelos de traducción

**C) Generator + Traducción**

- Traducción español → inglés: `translate_es_to_en_chunks(text)`

- Resumen en inglés: `summarize_bart(text)` usando `BART-large-CNN`

- Traducción inglés → español: `translate_en_to_es(text)`

- Función principal: `generate_rag_response_translate(query, retrieved_chunks)`

- Entrada: los chunks recuperados del Retriever + la pregunta

- Salida: respuesta coherente en español + lista de fuentes (`doc_id`)

- Prompt / control: se asegura que solo se use la información del contexto, se resuma y no se invente información



---



### Librerias

In [1]:
# Instalar librerias
!pip install pdfplumber
!pip install sentence-transformers
!pip install faiss-cpu


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install tf-keras


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [4]:
# Importar librerias
import pdfplumber
import os
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

c:\Users\sarit\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuración de carpetas

In [5]:
# Configuración de carpetas
RAW_FOLDER = "/content/data/raw"         # PDFs originales
PROCESSED_FOLDER = "/content/data/processed"  # Textos procesados


# Crear carpetas si no existen
os.makedirs(RAW_FOLDER, exist_ok=True)
os.makedirs(PROCESSED_FOLDER, exist_ok=True)

## **FASE I: RECUPERACIÓN DE INFORMACIÓN**

### Función para extraer texto de PDFs

Extrae el texto completo de cada PDF.

In [6]:
# Función para extraer texto de PDFs
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

### Convertir PDFs a texto y guardarlos

- Recorre todos los PDFs, extrae texto y guarda cada documento como .txt.

- Crea un diccionario con `doc_id`, `source`, `title` y `text`.

In [ ]:
# Convertir todos los PDFs a texto y guardarlos
documents = []
for filename in os.listdir(RAW_FOLDER):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(RAW_FOLDER, filename)
        text = extract_text_from_pdf(pdf_path)
        doc_id = filename.replace(".pdf", "")
        doc = {
            "doc_id": doc_id,
            "source": "BOE",
            "title": doc_id,
            "text": text
        }
        documents.append(doc)
        # Guardar cada texto en txt
        txt_path = os.path.join(PROCESSED_FOLDER, f"{doc_id}.txt")
        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(text)

print(f"Procesados {len(documents)} documentos.")

Procesados 11 documentos.


### Chuncking de documentos

- Divide cada documento en fragmentos de 1000 caracteres (chunks).

- Esto es necesario porque los LLMs no pueden procesar documentos muy largos de golpe.

In [8]:
# Chunking de documentos
def chunk_text(text, chunk_size=500):
    """Divide un texto en fragmentos de tamaño chunk_size (caracteres)"""
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

all_chunks = []
for doc in documents:
    chunks = chunk_text(doc["text"], chunk_size=1000)
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "doc_id": doc["doc_id"],
            "chunk_id": f"{doc['doc_id']}_chunk{i}",
            "text": chunk
        })

print(f"Total de chunks: {len(all_chunks)}")

Total de chunks: 1248


### Generar embeddings

Cada chunk se convierte en un vector semántico usando un modelo preentrenado (`all-MiniLM-L6-v2`).

In [9]:
# Generar embeddings con sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
texts = [c["text"] for c in all_chunks]
embeddings = model.encode(texts, convert_to_numpy=True)

print(f"Embeddings generados: {embeddings.shape}")

Embeddings generados: (1248, 384)


### Crear índices FAISS

- Construye un índice vectorial donde se almacenan todos los embeddings.

- Permite recuperar los chunks más relevantes dado un query.

In [10]:
# Crear índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print(f"FAISS index creado con {index.ntotal} vectores")

FAISS index creado con 1248 vectores


### Función de búsqueda

- Toma una pregunta, genera su embedding y busca los k chunks más similares en FAISS.

- Devuelve el texto y referencias de los documentos.

In [11]:
# Función de búsqueda
def retrieve(query, top_k=5):
    query_vec = model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        chunk = all_chunks[idx]
        results.append({"doc_id": chunk["doc_id"], "chunk_id": chunk["chunk_id"], "text": chunk["text"]})
    return results

Ejemplo de prueba

In [12]:
# Ejemplo de prueba
query = "¿Quién tiene derecho a asistencia sanitaria?"
results = retrieve(query, top_k=3)
for r in results:
    print("-------------------------------------------------------")
    print(r["doc_id"], r["chunk_id"])
    print(r["text"][:800], "...")

-------------------------------------------------------
BOE-A-2003-10715-consolidado BOE-A-2003-10715-consolidado_chunk53
bligado al pago de dicha asistencia.
c) Ser persona extranjera y con residencia legal y habitual en el territorio español y no
tener la obligación de acreditar la cobertura obligatoria de la prestación sanitaria por otra vía.
3. Aquellas personas que de acuerdo con el apartado 2 no tengan derecho a la
asistencia sanitaria con cargo a fondos públicos, podrán obtener dicha prestación mediante
el pago de la correspondiente contraprestación o cuota derivada de la suscripción de un
convenio especial.
4. Lo dispuesto en los apartados anteriores de este artículo no modifica el régimen de
asistencia sanitaria de las personas titulares o beneficiarias de los regímenes especiales
gestionados por la Mutualidad General de Funcionarios Civiles del Estado, la Mutualidad
General Judicial y el Instituto S ...
-------------------------------------------------------
BOE-A-2012-10477 

## **FASE II: GENERACIÓN DE TEXTO**

Criterios de la práctica:
- La respuesta debe basarse únicamente en los documentos recuperados.

- Si no hay información suficiente, debe indicar que no puede responder.

- Debe citar los documentos que sustentan la respuesta.

- Mantener el idioma de la pregunta.

### RAG con BART en inglés + traducción al español

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import MarianMTModel, MarianTokenizer

### Cargar modelo de embedding (Retriever)

`SentenceTransformer('all-MiniLM-L6-v2')`: crea embeddings (representaciones numéricas) de tus textos y de la consulta.

In [14]:
# Modelo para generar embeddings de chunks y consultas
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

### Crear función de recuperación (Retriever)

Convierte la pregunta del usuario en un vector (`query_vec`).

Busca los `top_k `chunks más similares en tu índice FAISS (`index`).

Devuelve una lista de chunks relevantes con su `doc_id`, `chunk_id` y texto.

Esto es el Retriever de RAG: busca los fragmentos de texto más relevantes.

In [15]:
def retrieve(query, all_chunks, index, top_k=5):
    query_vec = embedding_model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        chunk = all_chunks[idx]
        results.append({"doc_id": chunk["doc_id"], "chunk_id": chunk["chunk_id"], "text": chunk["text"]})
    return results

### Cargar modelo de lenguaje (LLM) open-source

BART: modelo de NLP para resumen de texto.

Se va a usar para generar un resumen de los documentos recuperados, que se traducen a inglés para obtener mejores resultados, y luego se traducen de vuelta al español.

In [16]:
# Configurar modelo BART (resumen en inglés)
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

### Configuración de modelos de traducción

Modelos MarianMT para traducir:

- Español → Inglés

- Inglés → Español

Esto permite generar resúmenes con BART (que funciona mejor en inglés) y luego volver al español.

In [17]:
pip install sentencepiece sacremoses

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# Configurar modelos de traducción
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

en_es_model_name = "Helsinki-NLP/opus-mt-en-es"
en_es_tokenizer = MarianTokenizer.from_pretrained(en_es_model_name)
en_es_model = MarianMTModel.from_pretrained(en_es_model_name)


c:\Users\sarit\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sarit\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-es-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fallin

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


### Funciones de traducción

- `translate_es_to_en`: traduce del español al inglés.

- `translate_en_to_es`: traduce del inglés al español.

También se hace uso de funciones para manejar chunks y traducir fragmentos:

- `translate_es_to_en_chunks`: traduce cada chunk por separado, luego une los resultados en un solo texto en inglés.

- Esto evita que los textos largos superen el límite de tokens del modelo de traducción.

In [19]:
# Funciones de traducción
def translate_es_to_en(text):
    inputs = es_en_tokenizer([text], return_tensors="pt", truncation=True, padding=True)
    translated = es_en_model.generate(**inputs, max_length=1024)
    return es_en_tokenizer.decode(translated[0], skip_special_tokens=True)

In [20]:
# Función para dividir texto en fragmentos
def chunk_text(text, chunk_size=500):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Función para traducir chunks de español a inglés
def translate_es_to_en_chunks(text):
    chunks = chunk_text(text, 500)  # dividir en trozos de 500 caracteres
    translated_chunks = [translate_es_to_en(c) for c in chunks]
    return " ".join(translated_chunks)


def translate_en_to_es(text):
    inputs = en_es_tokenizer([text], return_tensors="pt", truncation=True, padding=True)
    translated = en_es_model.generate(**inputs, max_length=512)
    return en_es_tokenizer.decode(translated[0], skip_special_tokens=True)

### Función para resumir con BART

- Recibe un texto (ya en inglés).

- Genera un resumen de hasta `max_output_tokens`.

- Usa beam search (`num_beams=4`) para obtener resúmenes más coherentes.

- Esto es el Generator de RAG.

In [87]:
# Función para generar resumen con BART
def summarize_bart(text, max_input_tokens=1024, max_output_tokens=200):
    inputs = bart_tokenizer.encode(text, return_tensors="pt", max_length=max_input_tokens, truncation=True)
    summary_ids = bart_model.generate(
        inputs,
        max_length=max_output_tokens,
        num_beams=4,
        early_stopping=True
    )
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

### Función principal RAG con traducción

Pasos completos del RAG traducido:

1. Recupera los chunks relevantes (Retriever).

2. Traduce el contexto a inglés (para BART).

3. Genera un resumen con BART.

4. Traduce el resumen a español.

5. Devuelve la respuesta final + las fuentes citadas.

In [88]:
def generate_rag_response_translate(query, retrieved_chunks):
    """
    Genera una respuesta traduciendo cada chunk individualmente para 
    respetar el límite de 512 tokens de MarianMT.
    """
    
    # Seleccionamos solo los top 3 o 4 para la generación (evita ruido y lentitud)
    # Aunque hayáis recuperado 8 para el Recall, BART trabaja mejor con 3-4 potentes
    chunks_to_process = retrieved_chunks[:3] 
    
    translated_context_list = []
    
    # PASO 1: Traducción individual de cada chunk (Evita el IndexError)
    for chunk in chunks_to_process:
        texto_es = chunk["text"]
        # Traducimos el fragmento de forma aislada
        texto_en = translate_es_to_en(texto_es) 
        translated_context_list.append(texto_en)
    
    # PASO 2: Unir el contexto ya traducido
    context_en = " ".join(translated_context_list)
    
    # PASO 3: Generar el resumen con BART (Límite 1024 tokens de entrada)
    #
    summary_en = summarize_bart(context_en)
    
    # PASO 4: Traducir la respuesta final al español
    respuesta_es = translate_en_to_es(summary_en)
    
    # PASO 5: Listar fuentes citadas (Requisito funcional)
    #
    fuentes = list(set([c["doc_id"] for c in chunks_to_process]))
    
    return respuesta_es, fuentes

Ejemplos de uso

In [89]:
# Query de ejemplo
query = "¿Quién tiene derecho a asistencia sanitaria?"

# Recuperar los chunks más relevantes
retrieved = retrieve(query, all_chunks, index, top_k=8)

# Generar respuesta RAG
respuesta, fuentes = generate_rag_response_translate(query, retrieved)

print("RESPUESTA RAG:")
print(respuesta)
print("\nFUENTES CITADAS:")
print(fuentes)

RESPUESTA RAG:
Todos los españoles, así como los extranjeros que hayan establecido su residencia en el territorio nacional, tienen derecho a la protección de la salud y la atención de la salud de conformidad con el artículo 1.2 de la Ley General de Salud 14/1986, de 25 de abril. Las prescripciones médicas utilizadas en la atención de la salud que se desarrollan fuera del ámbito hospitalario y no son las prescripciones médicas oficiales del Sistema Nacional de Salud, incluidas las de las mutualidades de funcionarios públicos, se seguirán en sus especificaciones técnicas.

FUENTES CITADAS:
['BOE-A-2012-10477', 'BOE-A-2011-1013-consolidado', 'BOE-A-2003-10715-consolidado']


In [24]:
query = "¿Qué requisitos deben cumplir los extranjeros para acceder a la asistencia sanitaria?"
retrieved = retrieve(query, all_chunks, index, top_k=8)
respuesta, fuentes = generate_rag_response_translate(query, retrieved)

print("RESPUESTA RAG:")
print(respuesta)
print("\nFUENTES CITADAS:")
print(fuentes)

RESPUESTA RAG:
Las Comunidades Autónomas establecerán el procedimiento de solicitud y expedición de un certificado que certifique a las personas extranjeras para poder recibir la asistencia a la que tienen derecho. Las personas que, de conformidad con el apartado 2, no tengan derecho a la asistencia sanitaria con cargo a fondos públicos, podrán obtener dicha prestación mediante el pago de la contrapartida o tasa correspondiente resultante de la firma de un acuerdo especial.

FUENTES CITADAS:
['BOE-A-2012-10477', 'BOE-A-1986-10499-consolidado', 'BOE-A-2011-1013-consolidado', 'BOE-A-2003-10715-consolidado']


In [25]:
query = "¿Qué medicamentos tienen precio de referencia según la normativa española?"
retrieved = retrieve(query, all_chunks, index, top_k=8)
respuesta, fuentes = generate_rag_response_translate(query, retrieved)

print("RESPUESTA RAG:")
print(respuesta)
print("\nFUENTES CITADAS:")
print(fuentes)

RESPUESTA RAG:
El precio de referencia será el importe máximo por el que se financiarán las presentaciones de los medicamentos incluidos en cada uno de los conjuntos que se determinen, siempre que se prescriban y distribuyan con cargo a fondos públicos; se establecerán los nuevos conjuntos y se revisarán anualmente los precios de los conjuntos existentes; el Ministerio de Salud regulará, por real decreto, un sistema de precios de los productos sanitarios.

FUENTES CITADAS:
['BOE-A-2015-8343-consolidado', 'BOE-A-2014-3189']


## **FASE III: Evaluación**

### **Medidas de evaluación**

**Recall@K**

- Qué mide: qué proporción de las fuentes correctas (ground_truth sources) aparece en los top K documentos recuperados por tu retriever.

- Rango: 0 a 1.

- Interpretación:

    - 1 → todas las fuentes correctas están entre los top K recuperados.

    - 0 → ninguna fuente correcta fue recuperada.

- Valor esperado en tu caso: como tu conjunto es pequeño (12 PDFs, 8 top_k), si tu retriever funciona bien deberías ver valores de 0.6 a 1.0 para preguntas sencillas.

**MRR (Mean Reciprocal Rank)**

- Qué mide: la posición de la primera fuente correcta en los documentos recuperados.

- Rango: 0 a 1.

- Interpretación:

    - 1 → la primera fuente correcta está en la primera posición.

    - 0.5 → la primera fuente correcta está en la segunda posición, etc.

-  esperado: depende de tu top_k y del retriever, pero valores >0.5 indican que la respuesta correcta aparece bastante arriba en la lista.

**BERTScore F1**

- Qué mide: similitud semántica entre la respuesta generada por el RAG y la respuesta esperada (ground_truth) usando embeddings de BERT.

- Rango: 0 a 1.

- Interpretación:

    - 1 → la respuesta generada es prácticamente igual semánticamente a la esperada.

    - 0 → la respuesta no tiene relación semántica.

- Valor esperado:

  - Si tus resúmenes RAG son coherentes pero no demasiado detallados, valores 0.6–0.85 son razonables.

  - Si la respuesta coincide literalmente con la ground truth, >0.9.

**FactScore**

- Qué mide: precisión del texto generado por el RAG en base a su coincidencia con los documentos originales utilizados para generar la respuesta.

- Rango: 0 a 1.

- Interpretación:

    - 1 → la respuesta generada coincide completamente con la información de los documentos originales.

    - 0 → la respuesta no refleja la información contenida en los documentos.

- Valor esperado:

    - Si las respuestas RAG son coherentes pero resumidas, valores de 0.6–0.85 son razonables.

    - Si la respuesta reproduce fielmente la información de los documentos, >0.9.

In [73]:
test_set = [
    # Documento: Ley 14/1986, de 25 de abril, General de Sanidad
    {
        'query': '¿Quiénes son los titulares del derecho a la protección de la salud según el artículo 1.2?',
        'expected_doc_ids': ['BOE-A-1986-10499-consolidado.pdf'],
        'ideal_answer': 'Todos los españoles y los ciudadanos extranjeros que tengan establecida su residencia en el territorio nacional.'
    },
    {
        'query': '¿A qué principios deben adecuar su funcionamiento los servicios sanitarios según el artículo siete?',
        'expected_doc_ids': ['BOE-A-1986-10499-consolidado.pdf'],
        'ideal_answer': 'A los principios de eficacia, celeridad, economía y flexibilidad.'
    },
    {
        'query': '¿Cómo se clasifican las infracciones sanitarias según la Ley 14/1986?',
        'expected_doc_ids': ['BOE-A-1986-10499-consolidado.pdf'],
        'ideal_answer': 'Se califican como leves, graves y muy graves.'
    },

    # Documento: Ley 41/2002 (Autonomía del Paciente)
    {
        'query': 'Defina "Consentimiento informado" según el artículo 3 de la Ley 41/2002.',
        'expected_doc_ids': ['BOE-A-2002-22188-consolidado.pdf'],
        'ideal_answer': 'Es la conformidad libre, voluntaria y consciente de un paciente, tras recibir información adecuada, para que tenga lugar una actuación que afecta a su salud.'
    },
    {
        'query': '¿En qué casos el consentimiento debe prestarse por escrito según el artículo 8.2?',
        'expected_doc_ids': ['BOE-A-2002-22188-consolidado.pdf'],
        'ideal_answer': 'En intervenciones quirúrgicas, procedimientos diagnósticos y terapéuticos invasores y procedimientos con riesgos de notoria repercusión negativa sobre la salud.'
    },
    {
        'query': '¿Qué obligaciones tiene el profesional sanitario según el artículo 2.6?',
        'expected_doc_ids': ['BOE-A-2002-22188-consolidado.pdf'],
        'ideal_answer': 'Prestación correcta de técnicas, cumplimiento de deberes de información y documentación clínica, y respeto a las decisiones libres del paciente.'
    },

    # Documento: Ley 16/2003 (Cohesión y Calidad)
    {
        'query': '¿Qué comprende el catálogo de prestaciones del SNS según el artículo 7.1?',
        'expected_doc_ids': ['BOE-A-2003-10715-consolidado.pdf'],
        'ideal_answer': 'Salud pública, atención primaria, especializada, sociosanitaria, urgencias, farmacia, ortoprótesis, productos dietéticos y transporte sanitario.'
    },
    {
        'query': '¿A quién corresponde la responsabilidad financiera de las prestaciones según el artículo 10.1?',
        'expected_doc_ids': ['BOE-A-2003-10715-consolidado.pdf'],
        'ideal_answer': 'A las comunidades autónomas, de conformidad con los acuerdos de transferencias y el sistema de financiación autonómica.'
    },
    {
        'query': 'Diferencia la cartera común básica de la suplementaria según los artículos 8 bis y 8 ter.',
        'expected_doc_ids': ['BOE-A-2003-10715-consolidado.pdf'],
        'ideal_answer': 'La básica incluye actividades asistenciales cubiertas completamente por financiación pública; la suplementaria incluye prestaciones ambulatorias sujetas a aportación del usuario.'
    },

    # Documento: Real Decreto 1718/2010 (Receta Médica)
    {
        'query': '¿Cuál es el plazo de validez de una receta médica en soporte papel según el artículo 5.5.b?',
        'expected_doc_ids': ['BOE-A-2011-1013-consolidado.pdf'],
        'ideal_answer': 'Diez días naturales a partir de la fecha de prescripción o de la fecha prevista para su dispensación.'
    },
    {
        'query': '¿Qué datos del prescriptor deben constar obligatoriamente en la receta según el artículo 3.2.c?',
        'expected_doc_ids': ['BOE-A-2011-1013-consolidado.pdf'],
        'ideal_answer': 'Nombre y apellidos, contacto directo (email y teléfono/fax), dirección profesional, cualificación, número de colegiado y firma.'
    },
    {
        'query': '¿Qué debe hacer el farmacéutico ante un error manifiesto en una receta electrónica (Art. 9.6)?',
        'expected_doc_ids': ['BOE-A-2011-1013-consolidado.pdf'],
        'ideal_answer': 'Puede bloquear cautelarmente la dispensación, comunicándolo telemáticamente al prescriptor e informando al paciente.'
    },

    # Documento: Real Decreto 1192/2012 (Asegurado y Beneficiario)
    {
        'query': '¿Qué organismo reconoce la condición de asegurado según el artículo 4.1?',
        'expected_doc_ids': ['BOE-A-2012-10477.pdf'],
        'ideal_answer': 'El Instituto Nacional de la Seguridad Social o, en su caso, el Instituto Social de la Marina.'
    },
    {
        'query': '¿Cuál es el límite de ingresos para ser asegurado según el artículo 2.1.b?',
        'expected_doc_ids': ['BOE-A-2012-10477.pdf'],
        'ideal_answer': 'No tener ingresos superiores en cómputo anual a cien mil euros.'
    },
    {
        'query': 'Enumere quiénes pueden ser beneficiarios de un asegurado según el artículo 3.1.',
        'expected_doc_ids': ['BOE-A-2012-10477.pdf'],
        'ideal_answer': 'Cónyuge o pareja de hecho, ex cónyuge a cargo con pensión compensatoria y descendientes menores de 26 años (o mayores con discapacidad ≥65%).'
    }
]

In [57]:
from sentence_transformers import util
# Este modelo es vital porque vuestras respuestas finales son en ESPAÑOL
model_eval = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def calculate_similarity(generated_text, reference_text):
    # 1. Dividimos en frases reales (por puntos)
    gen_sents = [s.strip() for s in generated_text.split('.') if len(s.strip()) > 10]
    ref_sents = [s.strip() for s in reference_text.split('.') if len(s.strip()) > 10]

    if not gen_sents or not ref_sents: return 0.0

    # 2. Generamos embeddings
    gen_embs = model_eval.encode(gen_sents, convert_to_tensor=True)
    ref_embs = model_eval.encode(ref_sents, convert_to_tensor=True)

    # 3. Calculamos la matriz de similitud de coseno
    cosine_scores = util.cos_sim(gen_embs, ref_embs)

    # 4. Lógica de "Mejor Pareja": Para cada frase generada, buscamos su máxima similitud
    max_scores = cosine_scores.max(dim=1)[0].cpu().numpy()

    return np.mean(max_scores)

In [74]:
import time
import numpy as np
from sentence_transformers import util

def run_full_evaluation(test_set, k=8):
    results = {
        "recall": [], "mrr": [], "bert_score": [], 
        "fact_score": [], "time": []
    }

    print(f"Iniciando evaluación sobre {len(test_set)} preguntas...")

    for item in test_set:
        query = item["query"]
        # Normalización correcta: quitamos .pdf para comparar
        expected_ids = [doc.replace(".pdf", "").strip() for doc in item["expected_doc_ids"]]
        ideal_ans = item["ideal_answer"]

        # --- MEDICIÓN DE TIEMPO ---
        start_time = time.time()

        # 1. Recuperación (Retrieval)
        retrieved = retrieve(query, all_chunks, index, top_k=k)
        # CORRECCIÓN DE TYPO: El .replace debe ir dentro de la comprensión de lista
        retrieved_ids = [c["doc_id"].replace(".pdf", "").strip() for c in retrieved]

        # 2. Generación (Generation)
        try:
            generated_ans, _ = generate_rag_response_translate(query, retrieved)
        except Exception as e:
            print(f"Error en generación para query '{query}': {e}")
            generated_ans = ""

        results["time"].append(time.time() - start_time)

        # --- CÁLCULO DE MÉTRICAS ---
        
        # A. Recall@K 
        found = any(eid in retrieved_ids for eid in expected_ids)
        results["recall"].append(1 if found else 0)

        # B. MRR (Mean Reciprocal Rank) 
        rank = 0
        for i, rid in enumerate(retrieved_ids):
            if rid in expected_ids:
                rank = 1 / (i + 1)
                break
        results["mrr"].append(rank)

        # C. BERTScore (Similitud con Ideal) [cite: 262]
        # Truncamos a 500 palabras para evitar el IndexError del modelo evaluador
        gen_truncated = " ".join(generated_ans.split()[:450])
        ideal_truncated = " ".join(ideal_ans.split()[:450])
        
        b_score = calculate_similarity(gen_truncated, ideal_truncated)
        results["bert_score"].append(b_score)

        # D. FactScore (Fidelidad al Contexto) 
        context_text = " ".join([c["text"] for c in retrieved])
        context_truncated = " ".join(context_text.split()[:450])
        
        f_score = calculate_similarity(gen_truncated, context_truncated)
        results["fact_score"].append(f_score)

    # --- MOSTRAR RESULTADOS ---
    print(f"\n--- RESULTADOS FINALES (Chatbot RAG) ---")
    print(f"Recall@{k}:     {np.mean(results['recall']):.4f} ")
    print(f"MRR:           {np.mean(results['mrr']):.4f} ")
    print(f"BERTScore:     {np.mean(results['bert_score']):.4f} [cite: 262]")
    print(f"FactScore:     {np.mean(results['fact_score']):.4f} [cite: 260]")
    print(f"Tiempo Medio:  {np.mean(results['time']):.2f} segundos ")

# Ejecutar la evaluación completa
run_full_evaluation(test_set, k=8)

Iniciando evaluación sobre 15 preguntas...
Error en generación para query 'Diferencia la cartera común básica de la suplementaria según los artículos 8 bis y 8 ter.': index out of range in self

--- RESULTADOS FINALES (Chatbot RAG) ---
Recall@8:     1.0000 
MRR:           0.8556 
BERTScore:     0.2838 [cite: 262]
FactScore:     0.7963 [cite: 260]
Tiempo Medio:  51.33 segundos 
